In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

# Definisci l'ordine dei mesi e anni
MESE_ANNO_ORDINE = ['Maggio_2023', 'Giugno_2023', 'Luglio_2023', 'Agosto_2023', 'Settembre_2023', 'Ottobre_2023',
                'Novembre_2023', 'Dicembre_2023', 'Gennaio_2024', 'Febbraio_2024', 'Marzo_2024',
                'Aprile_2024', 'Maggio_2024', 'Giugno_2024', 'Luglio_2024', 'Agosto_2024', 'Settembre_2024']

In [13]:
df = pd.read_csv('Lezioni.csv')#.drop(columns='Unnamed: 0')
#pd.DataFrame(df[(df['Index'] >= 269) & (df['Index'] <= 294)]['Professoressa'].value_counts().head(25))
df['MESE_ANNO'] = df['Mese'].astype(str) + '_' + df['Anno'].astype(str)
df.head()

,Index,Data,Ora,Professoressa,Livello,Lezione,Giorno della settimana,Mese,Anno,MESE_ANNO
0,1,2023-05-22,8,Virginia,B1,B132,Lunedì,Maggio,2023,Maggio_2023
1,2,2023-05-22,11,Graziana,B1,B135,Lunedì,Maggio,2023,Maggio_2023
2,3,2023-05-23,10,Francesca,C1,C123,Martedì,Maggio,2023,Maggio_2023
3,4,2023-05-23,11,Sara,B1,B143,Martedì,Maggio,2023,Maggio_2023
4,5,2023-05-24,10,Alice,B1,B141,Mercoledì,Maggio,2023,Maggio_2023


In [14]:
df.tail(1)['Giorno della settimana'].values[0]

'Giovedì'

In [15]:
df_ascending_false = df.sort_values(by='Data', ascending=False)
df_ascending_false.head()#.to_clipboard()

,Index,Data,Ora,Professoressa,Livello,Lezione,Giorno della settimana,Mese,Anno,MESE_ANNO
346,347,2024-09-12,10,Miriam,B2,B243,Giovedì,Settembre,2024,Settembre_2024
345,346,2024-09-11,9,Maddalena,C1,C119,Mercoledì,Settembre,2024,Settembre_2024
344,345,2024-09-11,7,Simona,C1,C118,Mercoledì,Settembre,2024,Settembre_2024
343,344,2024-09-10,9,Giulia,B1,B136,Martedì,Settembre,2024,Settembre_2024
342,343,2024-09-06,7,Chiara 2,C1,C122,Venerdì,Settembre,2024,Settembre_2024


In [16]:
df.describe().round().T

,count,mean,std,min,25%,50%,75%,max
Index,347.0,174.0,100.0,1.0,88.0,174.0,260.0,347.0
Ora,347.0,9.0,2.0,7.0,7.0,9.0,10.0,18.0
Anno,347.0,2023.0,0.0,2023.0,2023.0,2023.0,2024.0,2024.0


In [17]:
df.select_dtypes('object').describe()

,Data,Professoressa,Livello,Lezione,Giorno della settimana,Mese,MESE_ANNO
count,347,347,347,347,347,347,347
unique,251,67,3,102,7,12,17
top,2023-07-03,Federica,C1,C119,Lunedì,Giugno,Agosto_2023
freq,4,27,159,10,76,58,33


In [20]:
def ordina_in_base_al_periodo(dizionario_da_ordinare):
    dizionario_ordinato = {chiave: dizionario_da_ordinare[chiave] for chiave in MESE_ANNO_ORDINE if chiave in dizionario_da_ordinare}
    return dizionario_ordinato

dizionario_ordinato = ordina_in_base_al_periodo(df_ascending_false.groupby('MESE_ANNO')['Index'].count())
ora_media = ordina_in_base_al_periodo(round(df_ascending_false.groupby('MESE_ANNO')['Ora'].mean()))

In [19]:
def grafico_di_linea(list_dic, names, title, xaxis_title, yaxis_title, legend_title):
    # Crea il grafico a linee
    fig = go.Figure()

    # Itera attraverso i dizionari e i nomi
    for dic, name in zip(list_dic, names):
        fig.add_trace(go.Scatter(x=list(dic.keys()), y=list(dic.values()), 
                                 mode='lines+markers', name=name))

    # Aggiungi titolo e etichette agli assi
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        legend_title=legend_title,
        template='plotly_white',
        xaxis=dict(
            tickmode='linear'  # Forza la visualizzazione di tutti i mesi sull'asse X
        )
    )

    # Mostra il grafico
    fig.show()

# Esempio di chiamata alla funzione
grafico_di_linea(
    list_dic=[dizionario_ordinato, ora_media], 
    names=['Lezioni', "L'ora media"], 
    title='Quantità di Lezioni per periodi', 
    xaxis_title='Periodo', 
    yaxis_title='Quantità', 
    legend_title=''
)

In [ ]:
def distribuzione_delle_categorie(colonna:str):
    # Imposta l'ordine personalizzato per la colonna 'MESE_ANNO'
    df['MESE_ANNO'] = pd.Categorical(df['MESE_ANNO'], categories=MESE_ANNO_ORDINE, ordered=True)

    # Raggruppa i dati per livello e MESE_ANNO e conta il numero di lezioni
    data_grouped = df.groupby(['MESE_ANNO', colonna]).size().reset_index(name='Numero di Lezioni')

    # Crea il grafico a linee
    fig = px.line(data_grouped, x='MESE_ANNO', y='Numero di Lezioni', color=colonna, 
                title=f'Distribuzione delle Lezioni per {colonna} e Mese',
                labels={'MESE_ANNO': 'Mese e Anno', 'Numero di Lezioni': 'Numero di Lezioni'})

    # Aggiungi titolo e etichette agli assi
    fig.update_layout(
        template='plotly_white'
    )

    # Mostra il grafico
    fig.show()

distribuzione_delle_categorie('Livello')
df['Ora_cat'] = df['Ora'].astype('object')
distribuzione_delle_categorie('Ora_cat')

In [2]:
import plotly.figure_factory as ff
import pandas as pd

# Creazione del DataFrame dai dati CSV
data = {
    'Task': ['B132', 'B135', 'C123', 'B143'],  # Lezione
    'Start': ['2023-05-22', '2023-05-22', '2023-05-23', '2023-05-23'],  # Data
    'Finish': ['2023-05-23', '2023-05-23', '2023-05-24', '2023-05-24'],  # Data
    'Resource': ['B1', 'B1', 'C1', 'B1']  # Livello
}

df = pd.DataFrame(data)

# Creazione del diagramma di Gantt con Plotly
fig = ff.create_gantt(df, colors=['#7a0504', '#f55742', '#4e79a7'], index_col='Resource',
                      show_colorbar=True, group_tasks=True, showgrid_x=True, showgrid_y=True)

# Aggiunta di titoli e label per gli assi
fig.update_layout(
    title='Diagramma di Gantt delle Lezioni',
    xaxis_title='Data',
    yaxis_title='Lezione',
)

# Aggiunta di griglie e aggiustamenti di layout
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

# Personalizzazione del font e dello sfondo del grafico
fig.update_layout(
    font=dict(family='Arial', size=12, color='black'),
    plot_bgcolor='white',
)

# Visualizzazione del grafico
fig.show()

In [4]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
from collections import Counter

def differenza_giorni(dataframe):
    # Conversione della colonna 'Data' in datetime
    dataframe['Data'] = pd.to_datetime(dataframe['Data'])
    
    # Calcolo delle differenze in giorni tra le date consecutive
    dataframe = dataframe.sort_values('Data')
    dataframe['Differenza'] = dataframe['Data'].diff().dt.days
    
    # Calcolo della media e della differenza massima
    differenze = dataframe['Differenza'].dropna()
    media_differenza = round(differenze.mean(), 1)
    max_differenza = differenze.max()
    
    # Restituzione dei risultati
    testo = f'La media della differenza dei giorni tra le lezioni: {media_differenza} giorni. (Massimo: {max_differenza})'
    maggiori_differenze = differenze.tolist()
    maggiori_differenze.sort(reverse=True)
    
    return media_differenza, testo, maggiori_differenze

In [ ]:
differenza_giorni(df)

In [ ]:
def intervalloCV(dataframe, mese_anno):
    # Filtraggio per MESE_ANNO e conteggio delle differenze
    df_mese_anno = dataframe[dataframe['MESE_ANNO'] == mese_anno]
    differenze = differenza_giorni(df_mese_anno)[2]
    conteggio = Counter(differenze)
    
    chiave = sum(conteggio.keys())
    valore = sum(conteggio.values())
    
    return f'{mese_anno} -> {chiave} -- {valore}', valore - chiave, chiave, valore

# Filtraggio dei mesi e anni specifici
mesi_anno = df['MESE_ANNO'].unique().tolist()
intervalloCV_differenze = []
intervalloCV_chiavi = []
intervalloCV_valori = []

for mese_anno in mesi_anno:
    _, differenza, chiave, valore = intervalloCV(df, mese_anno)
    intervalloCV_differenze.append(differenza)
    intervalloCV_chiavi.append(chiave)
    intervalloCV_valori.append(valore)

# Creazione del grafico
fig = go.Figure()
fig.add_trace(go.Scatter(x=mesi_anno, y=intervalloCV_differenze, mode='lines', name='Differenza'))
fig.add_trace(go.Scatter(x=mesi_anno, y=intervalloCV_chiavi, mode='lines', name='Giorni'))
fig.add_trace(go.Scatter(x=mesi_anno, y=intervalloCV_valori, mode='lines', name='Ripetizione'))

fig.update_layout(
    title='IntervalloCV',
    xaxis=dict(title='Mese e Anno'),
    yaxis=dict(title='Punteggi'),
    template='plotly_white'
)

fig.show()

In [13]:
#df.groupby(['MESE_ANNO', 'Livello']).size().unstack().fillna(0).reset_index()

In [ ]:
"""#df = df.groupby(['MESE_ANNO', 'Livello']).size().unstack().fillna(0).reset_index()

# Creazione del grafico
fig = go.Figure()

for column in df.columns[1:]:  # Ignora la colonna 'MESE_ANNO'
    fig.add_trace(go.Scatter(x=df['MESE_ANNO'], y=df[column], mode='lines+markers', name=column))

# Impostazioni del layout
fig.update_layout(title='Quantità di Livelli per MESE_ANNO',
                  xaxis_title='MESE_ANNO',
                  yaxis_title='Quantità',
                  legend_title='Livelli',
                  template='plotly_white')

# Mostra il grafico
fig.show()"""

In [ ]:
"""def line_graphic(list_dic, eixo_X, eixo_y, colors, names_line):
    fig = go.Figure()

    for dic, color, name_line in zip(list_dic, colors, names_line):
        fig.add_trace(go.Scatter(x=list(dic.keys()), y=list(dic.values()), mode='lines', name=name_line, line=dict(color=color)))
        fig.update_layout(plot_bgcolor="white", xaxis_title=eixo_y, yaxis_title=eixo_X)

    fig.show() # lightgrey turquoise # deepskyblue violet lightseagreen lightpink"""

In [ ]:
ORDINE = df['MESE_ANNO'].to_list()
print(ORDINE)

In [ ]:
TOTALE = {chiave: df.groupby('MESE_ANNO')['Data'].count().sort_values(ascending=False).to_dict()[chiave] for chiave in ORDINE}
line_graphic([TOTALE], '', '', ['turquoise'], ['C1'])